In [2]:
from Import.NamedEntityVectorizer import NamedEntityVectorizer

In [27]:
import sys
del sys.modules['Import.NamedEntityVectorizer']
from Import.NamedEntityVectorizer import NamedEntityVectorizer
nerv = NamedEntityVectorizer(entity="PERSON",max_features=4)

texts = ["Donald Trump is made by Hillary Trump in London", "America's Fabian Retkowski is a person made by Donald Trump.", "Hillary Clinton is not a person, but Hillary Clinton is."]
            
nerv.fit(texts)
print(nerv.get_feature_names())
print(nerv.transform(texts))

['Donald Trump' 'Hillary Clinton' 'Fabian Retkowski' 'Hillary Trump']
[array([ 0.11111111,  0.        ,  0.        ,  0.11111111]), array([ 0.08333333,  0.        ,  0.08333333,  0.        ]), array([ 0.        ,  0.16666667,  0.        ,  0.        ])]


In [20]:
tagged = nerv._stanford_tagger(text="Donald Trump is made by Hillary Trump in London")
print(tagged)

[['PERSON', 'Donald'], ['PERSON', 'Trump'], ['O', 'is'], ['O', 'made'], ['O', 'by'], ['PERSON', 'Hillary'], ['PERSON', 'Trump'], ['O', 'in']]


In [23]:
list_of = [chunk[1] for chunk in tagged if chunk[0] == "PERSON"]
list_set_of = list(set(list_of))
print(list_of)
print(list_set_of)
for feature in list_of:
    try:
        if build_doc:
            doc_dict[feature] += 1
            vocabulary[feature] += 1
    except KeyError:
        vocabulary[feature] = 1
                    
    for feature in list_set_of:
        try:
            counter[feature] += 1
        except KeyError:
            counter[feature] = 1

print(list_of)
print(list_set_of)

['Donald', 'Trump', 'Hillary', 'Trump']
['Hillary', 'Trump', 'Donald']


NameError: name 'build_doc' is not defined

Hillary Trump | Donald Trump | Fabian Retkowski | Hillary Clinton

In [14]:
nerv = NamedEntityVectorizer(entity="PERSON",max_features=5)
texts = ["Max is sexy", "Aram is not sexy"]
nerv.fit(texts)
print(nerv.get_feature_names())
print(nerv.transform(texts))

[]
[array([], dtype=float64), array([], dtype=float64)]


In [5]:
nerv = NamedEntityVectorizer(entity="PERSON",max_features=4)
texts = ["ewe"]
nerv.fit(texts)
print(nerv.get_feature_names())
print(nerv.transform(texts))

[]
[array([], dtype=float64)]


In [6]:
tagged = [['Nets', 'ORGANIZATION'], ['Atlantic Division', 'ORGANIZATION'], ['Vince Carter', 'PERSON'], ['Toronto Raptors', 'ORGANIZATION'], ['Continental Arena', 'ORGANIZATION'], ['Jason Kidd', 'PERSON'], ['Kidd', 'PERSON'], ['Joumana', 'PERSON'], ['Nets', 'ORGANIZATION'], ['Raptors', 'ORGANIZATION'], ['Atlantic Division', 'ORGANIZATION'], ['Raptors', 'ORGANIZATION'], ['Carter', 'PERSON'], ['Toronto', 'LOCATION'], ['Raptors', 'ORGANIZATION'], ['Phoenix Suns', 'ORGANIZATION'], ['Andrea Bargnani', 'PERSON'], ['Raptors', 'ORGANIZATION'], ['Bargnani', 'PERSON'], ['Nets', 'ORGANIZATION'], ['Chris Bosh', 'PERSON'], ['Raptors', 'ORGANIZATION'], ['\nBosh', 'PERSON'], ['Toronto', 'LOCATION'], ['T. J. Ford', 'PERSON'], ['Mikki Moore', 'PERSON'], ['Bosh', 'PERSON'], ['Nets', 'ORGANIZATION'], ['Raptors', 'ORGANIZATION'], ['Nets', 'ORGANIZATION'], ['Raptors', 'ORGANIZATION'], ['Nets', 'ORGANIZATION'], ['Nets', 'ORGANIZATION'], ['Bargnani', 'PERSON'], ['Richard Jefferson', 'PERSON'], ['\nCarter', 'PERSON'], ['Nets', 'ORGANIZATION'], ['Raptors', 'ORGANIZATION'], ['Toronto', 'LOCATION'], ['Carter', 'PERSON'], ['Carter', 'PERSON'], ['Cliff Robinson', 'PERSON'], ['Carter', 'PERSON'], ['Robinson', 'PERSON'], ['Nets', 'ORGANIZATION'], ['Cavaliers', 'ORGANIZATION'], ['Cleveland', 'LOCATION'], ['Carter', 'PERSON'], ['Cliff Robinson', 'PERSON'], ['\nRobinson', 'PERSON'], ['Bulls', 'ORGANIZATION'], ['Nets', 'ORGANIZATION'], ['Josh Boone', 'PERSON']]
list_of = [chunk[0] for chunk in tagged if chunk[1] == "ORGANIZATION"]
list_set_of = list(set(list_of))
print(list_set_of)

['Bulls', 'Raptors', 'Nets', 'Continental Arena', 'Phoenix Suns', 'Toronto Raptors', 'Cavaliers', 'Atlantic Division']
